In [12]:
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import seaborn as sns            # Еще один пакет для графиков
import pickle                    # Пакет для подгрузки данных специфического для питона формата или для сохранения данных в файл

from wordcloud import WordCloud  # Пакет для построения облаков слов

plt.style.use('ggplot')          # задаём красивый стиль для графиков matplotlib-а
%matplotlib inline

In [13]:
# plt.rcParams['figure.figsize']=15,15
# topic_sim=[]
# for i in range(20):
#   v1 = ' '.join(topic_base[topic_base['topic']==i]['word_in topic'].values[:15])
#   v1_emb=embed_bert_cls(v1, model, tokenizer)
# #  print(v1)
#   for m in range(20):
#     sim_work=[]
#     v2 = ' '.join(topic_nmf[topic_nmf['topic']==m]['topic_word'].values[:15])
#     v2_emb=embed_bert_cls(v2, model, tokenizer)
#     sim=similarity(v1_emb, v2_emb)
#     sim_work.append(i)
#     sim_work.append(m)
#     sim_work.append(sim)
#     topic_sim.append(sim_work)
# #topic_sim
# topic_sim_ds=pd.DataFrame(topic_sim,columns=['base','nmf','cos_sim'])
# glue = topic_sim_ds.pivot(index='base', columns='nmf', values='cos_sim')
# sns.heatmap(glue, annot=True, fmt=".1f")

In [14]:
import sys
!{sys.executable} -m pip install --upgrade pymorphy2

from pymorphy2 import MorphAnalyzer

parser = MorphAnalyzer()

In [15]:
import requests
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

# Загрузка файла
url = "https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt"
response = requests.get(url)
rus_stops = response.text.splitlines()

punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'
filter_token = rus_stops + list(punctuation)

parser = MorphAnalyzer()

def preprocess_gemsim(input_text):
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter_token]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text if word]  # Проверка на пустые строки
    return lemmatized_text


<>:10: SyntaxWarning: invalid escape sequence '\]'
<>:10: SyntaxWarning: invalid escape sequence '\]'
C:\Users\Сократ\AppData\Local\Temp\ipykernel_19668\1493062947.py:10: SyntaxWarning: invalid escape sequence '\]'
  punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'


In [16]:
import pandas as pd

# Открытие файла с явным указанием кодировки
with open("../anek_utf8.txt", "r", encoding="utf-8") as file1:
    lines = []
    for line in file1:
        if line.strip():  # Проверка на пустую строку
            lines.append(line.strip())  # Удаление пробелов и символов новой строки

line_new = []
for i in range(len(lines) - 1):  # Используем 'i' вместо 'n'
    if lines[i][:15] == '<|startoftext|>':
        if lines[i + 1][:15] != '<|startoftext|>':
            line_new.append(lines[i][15:] + ' ' + lines[i + 1][15:])
        else:
            line_new.append(lines[i][15:])

# Создание DataFrame
ds_line = pd.DataFrame(line_new, columns=['text_base'])


In [17]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')#для табов

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [18]:
import nltk
nltk.download('punkt')

# ds_line["text_processed_new"] = ds_line["text_base"].apply(lambda text: preprocess_sklearn(text))
ds_line["text_processed"] = ds_line["text_base"].apply(lambda text: preprocess_gemsim(text))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сократ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
# Build the bigram  models
texts=(ds_line["text_processed"])
from gensim import corpora, models

bigram = models.Phrases(texts, min_count=3, threshold=5)
bigram_mod = models.phrases.Phraser(bigram)
texts_bi = make_bigrams(texts)
dictionary_bi = corpora.Dictionary(texts_bi)                 # составляем словарь из терминов с учетом биграмм слов предлжений
print('Размер словаря до фильтрации: {}'.format(len(dictionary_bi)))
dictionary_bi.filter_extremes(no_below=3, no_above=0.4, keep_n=3*10**6)
print('Размер словаря после фильтрации: {}'.format(len(dictionary_bi)))
corpus_bi = [dictionary_bi.doc2bow(text) for text in texts_bi]  # составляем корпус документов по словарю полученному с использованием биграмм

Размер словаря до фильтрации: 83955
Размер словаря после фильтрации: 37948


In [20]:
number_topics=20

In [21]:
def lda_model_gensim(corpus,dictionary,num_topics):
  ldamodel = models.ldamodel.LdaModel(
    corpus,
    id2word=dictionary,
    eval_every=20,
    num_topics=num_topics,
    passes=5
    )
  return ldamodel

In [22]:
ldamodel_bi=lda_model_gensim(corpus_bi,dictionary_bi,number_topics)

In [23]:
topics_gensim_bi = ldamodel_bi.show_topics(num_topics=number_topics, num_words=100,formatted=False)
topics_bi=pd.DataFrame(topics_gensim_bi)
topics_bi.columns=['topic num','topic_word']

In [24]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [25]:
vis_bi = gensimvis.prepare(ldamodel_bi, corpus_bi, dictionary_bi)

In [26]:
pyLDAvis.enable_notebook()

In [27]:
vis_bi

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16     0.155167  0.113911       1        1  6.775951
0      0.088810 -0.021139       2        1  6.246089
7     -0.310860  0.291463       3        1  6.093735
10     0.149183  0.058069       4        1  6.003983
19     0.130266  0.078119       5        1  5.757406
8      0.108738  0.077010       6        1  5.194471
12     0.060917  0.003172       7        1  5.191407
1      0.018383 -0.010042       8        1  5.014581
13     0.028119  0.043839       9        1  4.926211
2      0.064760  0.027222      10        1  4.905372
5     -0.008870 -0.021285      11        1  4.809885
18     0.008589 -0.016647      12        1  4.769579
15     0.049569  0.045224      13        1  4.676192
11    -0.062204 -0.188711      14        1  4.666243
6     -0.024394 -0.034131      15        1  4.662528
3     -0.061991 -0.158578      16        1  4.532819
9     -0.036323 -0.082166      17        1  4.066410
14    -0.080813 -0.097353      18        1  4.004191
4     -0.101750 -0.134330      19        1  3.951495
17    -0.175297  0.026354      20        1  3.751452, topic_info=              Term          Freq         Total Category  logprob  loglift
24              ``  19282.000000  19282.000000  Default  30.0000  30.0000
23              ''  17545.000000  17545.000000  Default  29.0000  29.0000
320           жена   9136.000000   9136.000000  Default  28.0000  28.0000
27         женщина   7102.000000   7102.000000  Default  27.0000  27.0000
193        девушка   5672.000000   5672.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
3552  когда-нибудь    319.674417    320.961422  Topic20  -4.8177   3.2790
1160        похоже    306.559021    307.846025  Topic20  -4.8596   3.2788
880          ответ    767.577457    939.390965  Topic20  -3.9417   3.0810
1279        писать    834.214253   1166.345825  Topic20  -3.8585   2.9479
74             ...    624.960873  39663.955827  Topic20  -4.1473  -0.8675

[733 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
23        3  0.999939       ''
2441     19  0.996869    ''_.-
74        1  0.147514      ...
74        2  0.094065      ...
74        3  0.005874      ...
...     ...       ...      ...
3187     12  0.997585  штирлиц
1079      1  0.997439  экзамен
1055      3  0.210351     язык
1055      8  0.787892     язык
1640      6  0.997979     яйцо

[897 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 1, 8, 11, 20, 9, 13, 2, 14, 3, 6, 19, 16, 12, 7, 4, 10, 15, 5, 18])

In [46]:
import requests
from pymorphy2 import MorphAnalyzer

# Загрузка файла со стоп-словами
url = "https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt"
response = requests.get(url)

# Проверка успешности загрузки
if response.status_code == 200:
    rus_stops = response.text.splitlines()  # Разделяем текст на строки
else:
    print("Ошибка при загрузке файла:", response.status_code)

# Определение знаков препинания
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—»«...–'
filter_token = rus_stops + list(punctuation)
stop_words=filter_token
# Инициализация морфологического анализатора
morph = MorphAnalyzer()

# Теперь вы можете использовать rus_stops и filter_token в вашем коде


In [51]:
import requests

# Загрузка файла со стоп-словами
url = "https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt"
response = requests.get(url)

# Проверка успешности загрузки
if response.status_code == 200:
    rus_stops = response.text.splitlines()  # Разделяем текст на строки
else:
    print("Ошибка при загрузке файла:", response.status_code)

# Определение знаков препинания
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—»«...–'
filter_token = rus_stops + list(punctuation)
filter = filter_token

# Теперь filter содержит список стоп-слов и знаков препинания


In [52]:
# Преобразуем каждый список в строку
ds_line["text_processed"] = ds_line["text_processed"].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Теперь можно снова выполнить подгонку векторизатора
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=filter)
tfidf = tfidf_vectorizer.fit_transform(ds_line["text_processed"].values.astype('U'))



tokens_sklearn = tfidf_vectorizer.get_feature_names_out()
from sklearn.feature_extraction.text import TfidfVectorizer#  TF-IDF (Term Frequency-Inverse Document Frequency).

no_topics=20
from sklearn.decomposition import NMF, LatentDirichletAllocation# Non-negative Matrix Factorization и Latent Dirichlet Allocation

nmf = NMF(n_components=no_topics, random_state=1024, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [53]:
plt.rcParams['figure.figsize']=15,15
topic_sim=[]
for i in range(20):
  v1 = ' '.join(topic_base[topic_base['topic']==i]['word_in topic'].values[:15])
  v1_emb=embed_bert_cls(v1, model, tokenizer)
#  print(v1)
  for m in range(20):
    sim_work=[]
    v2 = ' '.join(topic_nmf[topic_nmf['topic']==m]['topic_word'].values[:15])
    v2_emb=embed_bert_cls(v2, model, tokenizer)
    sim=similarity(v1_emb, v2_emb)
    sim_work.append(i)
    sim_work.append(m)
    sim_work.append(sim)
    topic_sim.append(sim_work)
#topic_sim
topic_sim_ds=pd.DataFrame(topic_sim,columns=['base','nmf','cos_sim'])
glue = topic_sim_ds.pivot(index='base', columns='nmf', values='cos_sim')
sns.heatmap(glue, annot=True, fmt=".1f")

NameError: name 'topic_base' is not defined